In [1]:
# Setup credentials

import canta_secret

consumer_key = canta_secret.key
consumer_secret = canta_secret.secret
bearer_token = canta_secret.bearer
access_token = canta_secret.access_token
access_secret = canta_secret.access_secret

In [2]:
# Authentication on Twitter API

import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if api.verify_credentials:
    print("Auth completed successfuly!")
else:
    print("Issue occoured during authentication")

Auth completed successfuly!


In [3]:
# Import serializer from an external module

from serializer import Serializer

In [4]:
# Get starting users info

accounts = ["damiano10"]
for account in accounts:
    serializer = Serializer(f'check_data/{account}')
    profile = api.get_user(account)
    profile_json = profile._json
    serializer.serialize_json(f'{account}_profile.json', profile_json)

Data serialized to path: check_data/damiano10/damiano10_profile.json


In [6]:
# Point #1 of the assignment: retrieve starting users followers and followings
accounts = ["damiano10"]
for account in accounts:
    print(f"Processing @{account}")
    serializer = Serializer(f'check_data/{account}')

### FOLLOWERS

    #account_followers = []
    #for item in tweepy.Cursor(
    #        api.followers,
    #        screen_name=account,
    #        skip_status=True,
    #        include_user_entities=False
    #).items():
    #    found_follower = item._json
    #    account_followers.append(found_follower)

    #print(f"Found {len(account_followers)} followers for @{account}")
    #serializer.serialize_json(f"{account}_follower.json", account_followers)

### FOLLOWINGS

    account_followings = []
    for item in tweepy.Cursor(
            api.friends,
            screen_name=account,
            skip_status=True,
            include_user_entities=False
    ).items():
        found_followings = item._json
        account_followings.append(found_followings)

    print(f"@{account} follows {len(account_followings)} users")
    serializer.serialize_json(f"{account}_following.json", account_followings)

Processing @damiano10


Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 893


@damiano10 follows 840 users
Data serialized to path: check_data/damiano10/damiano10_following.json


In [16]:
# Points #2 and #3 of the assignment: pick 5 random followers of the starting users,
# retrieve 10 followers each, pick 5 random followings of the starting users
# and retrieve 10 followings each

import random

for account in accounts:
    serializer = Serializer(f'check_data/{account}')
    json = serializer.read_json(f"{account}_follower.json")
    for count in range(0, 5):
        random_follower = random.choice(json)
        random_follower_screenName = random_follower["screen_name"]
        random_follower_id = random_follower["id"]
        random_follower_followers = []
        for item in tweepy.Cursor(
                api.followers,
                screen_name=random_follower_screenName,
                skip_status=True,
                include_user_entities=False
        ).items(10):
            found_follower = item._json
            random_follower_followers.append(found_follower)
        print(f"Found {len(random_follower_followers)} followers for @{random_follower_screenName}")
        serializer.serialize_json(f"random_{random_follower_id}_follower.json", random_follower_followers)

    json = serializer.read_json(f"{account}_following.json")
    for count in range(0, 5):
        random_following = random.choice(json)
        random_following_screenName = random_following["screen_name"]
        random_following_id = random_following["id"]
        random_following_followings = []
        for item in tweepy.Cursor(
                api.friends,
                screen_name=random_following_screenName,
                skip_status=True,
                include_user_entities=False
        ).items(10):
            found_friend = item._json
            random_following_followings.append(found_friend)
        print(f"@{random_following_screenName} follows {len(random_following_followings)} users")
        serializer.serialize_json(f"random_{random_following_id}_following.json", random_following_followings)

Data read from path: check_data/damiano10/damiano10_follower.json


Rate limit reached. Sleeping for: 834


Found 10 followers for @alharth_ahmed
Data serialized to path: check_data/damiano10/random_2492690671_follower.json
Found 10 followers for @rober_centeno
Data serialized to path: check_data/damiano10/random_190821474_follower.json
Found 10 followers for @ijaitjournal
Data serialized to path: check_data/damiano10/random_761153954809913344_follower.json
Found 10 followers for @LlorenteyCuenca
Data serialized to path: check_data/damiano10/random_50257296_follower.json
Found 10 followers for @hmuehlburger
Data serialized to path: check_data/damiano10/random_73705954_follower.json
Data read from path: check_data/damiano10/damiano10_following.json
@murefil follows 10 users
Data serialized to path: check_data/damiano10/random_124319949_following.json
@ecir2017 follows 10 users
Data serialized to path: check_data/damiano10/random_717348489215262724_following.json
@bpoblete follows 10 users
Data serialized to path: check_data/damiano10/random_87985586_following.json
@FvaGc follows 10 users
Data

In [20]:
import os
s = Serializer(f"check_data/{account}")
profs = ["Miccighel_", "damiano10", "eglu81", "mizzaro", "KevinRoitero"]
with os.scandir(f"check_data/{account}") as directory:
    for entry in directory:
        if "random" in entry.name:
            random_group = s.read_json(entry.name)
            for user in random_group:
                if user["screen_name"] in profs:
                    print(f"Conflict found in file {entry.name}")

Data read from path: check_data/damiano10/random_761153954809913344_follower.json
Data read from path: check_data/damiano10/random_190821474_follower.json
Data read from path: check_data/damiano10/random_16423883_following.json
Data read from path: check_data/damiano10/random_73705954_follower.json
Data read from path: check_data/damiano10/random_124319949_following.json
Data read from path: check_data/damiano10/random_87985586_following.json
Data read from path: check_data/damiano10/random_2492690671_follower.json
Data read from path: check_data/damiano10/random_717348489215262724_following.json
Data read from path: check_data/damiano10/random_313207740_following.json
Data read from path: check_data/damiano10/random_50257296_follower.json


In [21]:
# Point #4 of the assignment: retrieve all encountered users' profile
from datetime import datetime
import os

error_count = 0         # Keep trace of how many errors occurred during user retrieval (account not found)
duplicate_count = 0     # Keep trace of users already encountered

all_users = []
processed_ids = []

print(f"Start at {datetime.now()}")
for account in accounts:
    print(
        f'\n\n*************************************\nProcessing {account} and his friends\n*************************************')
    serializer = Serializer(f'check_data/{account}')
    with os.scandir(f'check_data/{account}') as it:
        for entry in it:
            if entry.name.endswith('.json') and not entry.name.endswith('profile.json'):
                print('\n\n******************')
                users_data = serializer.read_json(f"{entry.name}")
                print(f'\nProcessing {entry.name}, containing {len(users_data)} users\n******************\n\n')
                for user in users_data:
                    if user["id"] not in processed_ids:
                        try:
                            print(f'Processing {user["id"]}, user #{len(all_users) + 1}')
                            user_details = api.get_user(user["id"])._json
                            useful_user_details = {
                                "id": user_details["id"],
                                "name": user_details["name"],
                                "screen_name": user_details["screen_name"],
                                "description": user_details["description"],
                                "followers_count": user_details["followers_count"],
                                "friends_count": user_details["friends_count"],
                                "profile_image_url_https": user_details["profile_image_url_https"]
                            }
                            all_users.append(useful_user_details)
                            processed_ids.append(user_details["id"])
                        except tweepy.TweepError:
                            error_count += 1
                            print("Skipped user because of error")
                    else:
                        duplicate_count += 1
serializer = Serializer('check_data')
print('\n\n*************************************\n')
serializer.serialize_json(f"damiano10_all_users.json", all_users)
print('\n*************************************\n\n')
print(f'Found {error_count} errors and {duplicate_count} duplicates')

Start at 2020-11-16 22:23:43.441476


*************************************
Processing damiano10 and his friends
*************************************


******************
Data read from path: check_data/damiano10/random_761153954809913344_follower.json

Processing random_761153954809913344_follower.json, containing 10 users
******************


Processing 14616323, user #1
Processing 288719447, user #2
Processing 1234838196028088320, user #3
Processing 1020592502674345984, user #4
Processing 1173898068791562240, user #5
Processing 1024563318697021440, user #6
Processing 1040034655544598528, user #7
Processing 410082141, user #8
Processing 1108360423794003968, user #9
Processing 166533062, user #10


******************
Data read from path: check_data/damiano10/random_190821474_follower.json

Processing random_190821474_follower.json, containing 10 users
******************


Processing 195065263, user #11
Processing 604217502, user #12
Processing 2665463588, user #13
Processing 30487354

Rate limit reached. Sleeping for: 476


Processing 1280838371682779136, user #902
Processing 812055605510803456, user #903
Processing 11773812, user #904
Processing 948669952843595776, user #905
Processing 1168926603855958017, user #906
Processing 484733763, user #907
Processing 1228678952, user #908
Processing 160246849, user #909
Processing 1251059396991311872, user #910
Processing 1251108504653361152, user #911
Processing 1251034648492138498, user #912
Processing 1251232344947613701, user #913
Processing 1251073078211178496, user #914
Processing 1077734311799984128, user #915
Processing 101818541, user #916
Processing 1148595838664552451, user #917
Processing 251614179, user #918
Processing 1238826270, user #919
Processing 755513278625898496, user #920
Processing 2522143308, user #921
Processing 114146587, user #922
Processing 1605951265, user #923
Processing 38048603, user #924
Processing 906922846412267521, user #925
Processing 918943909811769344, user #926
Processing 47681517, user #927
Processing 1204339794471538688, 

In [ ]:
# As requested, before building the social network is necessary to check friendships

def get_friendship(sourceid, targetid, api):
    kind = ""

    friendship = api.show_friendship(source_id=sourceid, target_id=targetid)

    if not friendship[0].following and not friendship[0].followed_by:
        kind = "none"
    elif not friendship[0].following and friendship[0].followed_by:
        kind = "r_l"
    elif friendship[0].following and not friendship[0].followed_by:
        kind = "l_r"
    else:
        kind = "bi"

    return {
        "source_id": sourceid,
        "target_id": targetid,
        "friendship": kind
    }

serializer = Serializer('check_data')
users = serializer.read_json("all_users.json")
edges = []
count = 0
for account in accounts:
    serializer = Serializer(f'check_data/{account}')
    account_json = serializer.read_json(f"{account}_profile.json")
    account_id = account_json["id"]
    for user in users:
        if user["id"] is not account_id:
            edges.append(get_friendship(account_id, user["id"], api))
            count += 1
            print(f"Added friendship between {account} and {user['screen_name']} #{count}")
    # There's also the necessity to check friendships between the random picked users at Point #2 and #3 and their followers and followings
    with os.scandir(f'check_data/{account}') as it:
        for entry in it:
            if entry.name.endswith('.json') and entry.name.startswith('random'):
                fileId = int(entry.name.split("_")[1])
                json = serializer.read_json(entry.name)
                for profile in json:
                    edges.append(get_friendship(fileId, profile["id"], api))
                    count += 1
                    print(f"Added friendship between {fileId} and {profile['screen_name']} #{count}")

serializer = Serializer('check_data')
serializer.serialize_json(f'all_friendships.json', edges)

In [ ]:
# Point #5 of the assignment: build the social network